## Напишем через рандом нумпая и классы
Используем список, генерируемый в начале работы через numpy rd.randint(0, 33554432, 262144)

In [2]:
from numpy import random as rd
import numpy as np
import matplotlib.pyplot as plt
import TtR_Germany.ipynb as TG


deck_names = {"green":0, "blue":1, "white":2, "yellow":3, "red":4, "black":5, "pink":6, "orange":7, "locomotive":8, "gray": 9}
deck_names_inv = ["green", "blue", "white", "yellow", "red", "black", "pink", "orange", "locomotive", "gray"]
decks, deck_num = [], 0

class Card():
    def __init__(self, vagon_type):
        if vagon_type in deck_names_inv:
            self.strtype = vagon_type
            self.inttype = deck_names[vagon_type]
    
    def __str__(self):
        return self.strtype
    
    def __int__(self):
        return self.inttype


def deck_generate(size = 2**20, max_val = 2**27) -> list:
    return rd.randint(0, max_val, (size,110))


def get_card_from_deck(deck, target) -> None: #[12,12,12,12,12,12,12,12,14]
    num = decks[deck_num]%np.sum(deck)
    for i in range(len(deck)):
        num -= deck[i]
        if num < 0: 
            deck[i] -= 1
            target[i] += 1

        
def count_cards(need_type, target) -> int:
    if need_type != "gray":
        return target[deck_names[need_type]]
    max_cnt = 0
    for i in range(9):
        if target[deck_names[i]] > max_cnt: max_cnt = target[deck_names[i]]
    return max_cnt
    

def refresh_table(deck, deck_num, table) -> None:
    while(np.sum(table) != 5):
        get_card_from_deck(deck, deck_num, table)
        if count_cards("locomotive", table) == 3: table = np.zeros(9)


class Player():
    
    def __init__(self, need_type = "gray", need_cnt = 1):
        self.holded = np.zeros(9)
        self.ntype = need_type
        self.ncnt = need_cnt
    
    def get_size(self):
        return np.sum(self.holded)

    size = property(get_size)
    
    def __getcardfromtable(self):
        pass
    
    def GetCard(self, nedd_type):
        pass
    
    

#  deck_generate()


ModuleNotFoundError: No module named 'TtR_Germany'

In [28]:
Misha = Player()
Misha.holded

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
%%time
decks = deck_generate()
print(decks[0])

[ 51965973 119516148  65219561 112464209 107173343 111417247  17764018
 132043686 131472822 114635127 122762638  12812658  20672881  69580429
 114313544 122038436  98787837  45650046  56617311 125569033  23211340
  80143366  58851030  54544457  36979665  34762024  65234552  79660291
  41087535 123543854 133107851   5895769  15141986  62981787  29257149
 129367671  40857656 125117123  34623663  44762718  72386591 103518135
 109194335   8832029  83521839 115098383   7393972  74062218   1951871
  83795325 106481294 121316099 102296462  76418584  35816583   9563145
  99184193  74114451  92841833  82246151  96850172 124341500  77708398
 102535372  30579898   6816914  90783285  66194619 117023427  25899188
 111987473 101256212  31229153 119006714  35530177 104080073 131685658
  55483798  11427035   1391524 110887446  54554776  59665369  68464999
  60532950 100882478  58186724  26909088  17138584   8633798  60046195
  24724910  34292515  72755043 110409167  32834347 131408139  16364079
  8592

In [ ]:
%%time

#Parametrs
need_type = "blue"
need_cnt = 4
Num_of_launches = 2 * 10**4
CPT = 2
ProgressBar = True
Get_Locomotives = "weak"

#Code
NOL = Num_of_launches
PB = ProgressBar
if NOL<100: PB = False
GL = Get_Locomotives

# fig, m_axs = plt.subplots(1, 2, figsize=(8.4*2, 1*1.8*2), constrained_layout=False)

def Turns_for_road_table_np(need_type, need_cnt, ProgressBar = PB, Num_of_launches = NOL,\
                         cards_per_turn = CPT, Get_Locomotives = GL) -> float:
    deck = nrd.randint(0, 33554432, 262144)
    turns = np.empty((Num_of_launches))

    for i in range(Num_of_launches):
        if ProgressBar: 
            if (i+1)%(Num_of_launches//10) == 0: print((i+1)//(Num_of_launches//100), end = " ")
        ct = count_turns_table_np(need_type, need_cnt, cards_per_turn, Get_Locomotives)
        turns[i] = ct
    
    if ProgressBar: print("\n")
        
#     fig, m_axs = plt.subplots(1, 1, figsize=(18, 7))   
#     m_axs.plot(np.linspace(0,Num_of_launches-1,Num_of_launches),turns)
#     plt.show()
    
    return np.average(turns)

print(Turns_for_road_table(need_type, need_cnt, Num_of_launches=NOL))


## Ну а тут другая реализация

In [50]:
import numpy as np
import matplotlib.pyplot as plt

deck_names = {"green":0, "blue":1, "white":2, "yellow":3, "red":4, "black":5, "pink":6, "orange":7, "locomotive":8, "gray": 9}
LOC_COST = 2

def deck_generate() -> list:
    deck = [i for i in range(9)]*12
    deck.append(8)
    deck.append(8)
    return deck
    
    
def deck_shuffle(deck) -> None:
    np.random.shuffle(deck)

    
    
def get_card_from_deck(deck, target) -> None:
    target.append(deck[0])
    deck.pop(0)
    
    
def get_card_from_table(deck, table, target, card_type) -> None:
    for crd_idx, crd in enumerate(table):
        if crd == card_type:
            target.append(crd)
            table.pop(crd_idx)
            refresh_table(deck, table)
            return

def count_type(target, need_type) -> int:
    cnt = 0
    for crd in target:
        if crd == need_type: cnt +=1
    return cnt

    
def refresh_table(deck, table) -> None:
    while len(table) != 5:
        get_card_from_deck(deck, table)
        if count_type(table, 8) == 3: table = []


def count_turns(deck, need_type = 9, need_cnt = 1, GL = "strong", CPT = 2) -> int:
    player = []
    table = []
    refresh_table(deck, table)
    turns_cnt = 0
    
    if need_type == 9:
        break_flag = False
        if GL == "strong":
            card_counts = [0]*8
            while 1:
                locs = count_type(player, 8)
                for i in range(8):
                    card_counts[i] = count_type(player, i)
                    if card_counts[i] + locs >= need_cnt:
                        break_flag = True
                        break
                if break_flag:
                    break
                turns_cnt += 1
                # вот тут должна быть оценка того, какую карту лучше всего взять
                for i in range(CPT):
                    if count_type(table, np.argmax(card_counts)):
                        get_card_from_table(deck, table, player, np.argmax(card_counts))
                    elif count_type(table, 8) and CPT-i >= LOC_COST:
                        get_card_from_table(deck, table, player, 8)
                    else:
                        get_card_from_deck(deck, player)
                
        elif GL == "weak":
            card_counts = [0]*8
            while 1:
                locs = count_type(player, 8)
                for i in range(8):
                    card_counts[i] = count_type(player, i)
                    if card_counts[i] + locs >= need_cnt:
                        break_flag = True
                        break
                if break_flag:
                    break
                turns_cnt += 1
                # вот тут должна быть оценка того, какую карту лучше всего взять
                for i in range(CPT):
                    if count_type(table, np.argmax(card_counts)):
                        get_card_from_table(deck, table, player, np.argmax(card_counts))
                    elif count_type(table, 8) and need_cnt - np.argmax(card_counts) <=1 and CPT-i >= LOC_COST:
                        get_card_from_table(deck, table, player, 8)
                    else:
                        get_card_from_deck(deck, player)
                
        elif GL == "never":
            card_counts = [0]*8
            while 1:
                locs = count_type(player, 8)
                for i in range(8):
                    card_counts[i] = count_type(player, i)
                    if card_counts[i] + locs >= need_cnt:
                        break_flag = True
                        break
                if break_flag:
                    break
                turns_cnt += 1
                # вот тут должна быть оценка того, какую карту лучше всего взять
                for i in range(CPT):
                    if count_type(table, np.argmax(card_counts)):
                        get_card_from_table(deck, table, player, np.argmax(card_counts))
                    else:
                        get_card_from_deck(deck, player)
                
        else:
            return -1
        
        return turns_cnt
        
        
    else:
        if GL == "strong":
            while 1:
                locs = count_type(player, 8)
                if count_type(player, need_type) + locs >= need_cnt:
                    break
                turns_cnt += 1
                # вот тут должна быть оценка того, какую карту лучше всего взять
                for i in range(CPT):
                    if count_type(table, need_type):
                        get_card_from_table(deck, table, player, nneed_type)
                    elif count_type(table, 8) and CPT-i >= LOC_COST:
                        get_card_from_table(deck, table, player, 8)
                    else:
                        get_card_from_deck(deck, player)
                
        elif GL == "weak":
            while 1:
                locs = count_type(player, 8)
                if count_type(player, need_type) + locs >= need_cnt:
                    break
                turns_cnt += 1
                # вот тут должна быть оценка того, какую карту лучше всего взять
                for i in range(CPT):
                    if count_type(table, need_type):
                        get_card_from_table(deck, table, player, need_type)
                    elif count_type(table, 8) and (need_cnt - need_type - locs) <=1 and CPT-i >= LOC_COST:
                        get_card_from_table(deck, table, player, 8)
                    else:
                        get_card_from_deck(deck, player)
            
        elif GL == "never":
            while 1:
                locs = count_type(player, 8)
                if count_type(player, need_type) + locs >= need_cnt:
                    break
                turns_cnt += 1
                # вот тут должна быть оценка того, какую карту лучше всего взять
                for i in range(CPT):
                    if count_type(table, need_type):
                        get_card_from_table(deck, table, player, need_type)
                    else:
                        get_card_from_deck(deck, player)
        
        
        else:
            return -1
        
        return turns_cnt

    


def Turns_for_road(need_type, need_cnt, ProgressBar = PB, Num_of_launches = NOL,\
                         cards_per_turn = CPT, Get_Locomotives = GL) -> float:
    decks = []
    for i in range(NOL):
        dc = deck_generate()
        deck_shuffle(dc)
        decks.append(dc)

    if PB: 
        pb_list = list(map(int, np.linspace(0, NOL-1, 11)))
        pb_parts = np.linspace(0, 100, 11)
    
    turns = []
    for i in range(NOL):
        if PB:
            if i in pb_list: 
                print(int(pb_parts[0]), end = " ")
                pb_parts = pb_parts[1:]
        turns.append(count_turns(decks[i], nt, need_cnt, GL, CPT))
    
    if PB: print("\n")
    return np.mean(turns)
    

## Разовый подсчёт
need_type - цвет перегона, на английском ("green", "blue", "white", "yellow", "red", "black", "pink", "orange", "gray")\
need_cnt - длина рассматриваемого перегона\
Num_of_launches - количество прогонов случая\
Cards_per_turn  - количество карт, которые берутся в ход.\
ProgressBar - Показ статуса подсчёта\
Get_Locomotives - будем ли мы брать локомотивы со стола ("never", "weak", "strong")\
    - never - вообще не берём локомотивы со стола\
    - weak - берём локомотив со стола только если он завершает наш набор\
    - strong - берём локомотив со стола, если там нет вагона нужного цвета

In [55]:
%%time

#Parametrs
need_type = "pink"
need_cnt = 4
Num_of_launches = 2 * 10**4
ProgressBar = True
Get_Locomotives = "never"
Cards_per_turn = 2

#Code
nt = deck_names[need_type]
NOL = Num_of_launches
PB = ProgressBar
if NOL<100: PB = False
GL = Get_Locomotives
CPT = Cards_per_turn

ANoT = Turns_for_road(nt, need_cnt)
print(f'\nAverage number of turns: {ANoT}')


0 10 20 30 40 50 60 70 80 90 100 


Average number of turns: 7.49965
Wall time: 728 ms


## Большой подсчёт

In [64]:
%%time

#Parametrs
Num_of_launches = 2 * 10**4
ProgressBar = False
Get_Locomotives = "never"
Cards_per_turn = 2

#Code
NOL = Num_of_launches
PB = ProgressBar
if NOL<100: PB = False
GL = Get_Locomotives
CPT = Cards_per_turn


# colors = ("green", "blue", "white", "yellow", "red", "black", "pink", "orange", "gray")
# ANoT = np.array([np.nan for _ in range(63)], dtype=float).reshape(9,7) # h - color; w - len

# for clr_idx, clr in enumerate(colors):
#     nt = deck_names[clr]
#     print(clr, end = " ")
#     for way_len in range(1,8):
#         Tfr = Turns_for_road(nt, way_len)
#         ANoT[clr_idx][way_len-1] = Tfr

# print('\n')    
# print(ANoT)


print(np.mean(ANoT[:8], axis=0))


[ 1.7220125  3.3966     5.4588625  7.490275   9.537725  11.5936375
 13.6601125]
Wall time: 0 ns
